In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
listings=pd.read_csv("/content/drive/My Drive/BT4222_Project/final_datasets/listings_final_version.csv")
customers=pd.read_csv("/content/drive/My Drive/BT4222_Project/final_datasets/customers_final_version.csv")

In [4]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12222 entries, 0 to 12221
Data columns (total 68 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              12222 non-null  int64  
 1   description                     12222 non-null  object 
 2   host_since                      12222 non-null  object 
 3   host_is_superhost               12222 non-null  int64  
 4   latitude                        12222 non-null  float64
 5   longitude                       12222 non-null  float64
 6   accommodates                    12222 non-null  int64  
 7   bathrooms                       12222 non-null  float64
 8   bedrooms                        12222 non-null  float64
 9   beds                            12222 non-null  float64
 10  amenities                       12222 non-null  object 
 11  price                           12222 non-null  float64
 12  minimum_nights                  

In [5]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130602 entries, 0 to 130601
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   reviewer_id                  130602 non-null  int64  
 1   listing_id                   130602 non-null  int64  
 2   date                         130602 non-null  object 
 3   comments                     130602 non-null  object 
 4   price                        130602 non-null  float64
 5   review_scores_rating         130602 non-null  float64
 6   review_scores_accuracy       130602 non-null  float64
 7   review_scores_cleanliness    130602 non-null  float64
 8   review_scores_checkin        130602 non-null  float64
 9   review_scores_communication  130602 non-null  float64
 10  review_scores_location       130602 non-null  float64
 11  review_scores_value          130602 non-null  float64
 12  polarity_score               130602 non-null  float64
 13 

# Knowledge Recommender

In [6]:
import pandas as pd
import numpy as np

In [7]:
df_hr = customers.copy()
df_hr['overall_review'] = (df_hr.review_scores_rating/100 + (df_hr.review_scores_accuracy + df_hr.review_scores_cleanliness + df_hr.review_scores_checkin + \
df_hr.review_scores_communication + df_hr.review_scores_location + df_hr.review_scores_value)/10)/7
df_hr = df_hr.drop(columns = ['review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value'])

In [8]:
df_hr['preference'] = (df_hr['polarity_score']+1)*df_hr['overall_review']/2
df_hr = df_hr.drop(columns = ['date','comments','polarity_score','overall_review','sentiment'])

In [9]:
table_customer = pd.pivot_table(df_hr,index='listing_id')

In [10]:
table_customer.describe()

,preference,price,reviewer_id
count,8154.000000,8154.000000,8.154000e+03
mean,0.792432,146.504460,1.114148e+08
std,0.164599,225.803786,6.839374e+07
min,0.001640,10.800000,3.538400e+04
25%,0.738336,62.333333,6.299261e+07
50%,0.831885,110.000000,1.000677e+08
75%,0.897666,180.000000,1.448609e+08
max,0.999550,9999.000000,3.829120e+08


In [11]:
preference_dict = {}
for index, data in table_customer.iterrows():
  preference_dict[index] = data[0]

In [12]:
listings['preference'] = listings["id"].apply(lambda x: preference_dict[x] if x in preference_dict.keys() else 0.792432) #if not found in customers then use the mean value as preference

In [13]:
df_kr = listings[['host_is_superhost', 'nearby_sites_rating', 'bathrooms', 'bedrooms', 'beds', 'instant_bookable','amenities_list', 'price', 'preference']]

In [15]:
df_kr.head()

,host_is_superhost,nearby_sites_rating,bathrooms,bedrooms,beds,instant_bookable,amenities_list,price,preference
0,0,1.135049,1.0,1.0,1.0,0,"['TV', 'Cable TV', 'Wifi', 'Air conditioning',...",200.0,0.894053
1,0,1.288425,1.0,1.0,1.0,0,"['TV', 'Cable TV', 'Internet', 'Wifi', 'Air co...",76.0,0.893712
2,0,1.982270,1.0,1.0,1.0,0,"['Internet', 'Wifi', 'Air conditioning', 'Pool...",80.0,0.792432
3,0,2.550789,1.0,0.0,3.0,0,"['TV', 'Internet', 'Wifi', 'Air conditioning',...",143.0,0.817946
4,0,1.689570,1.0,1.0,1.0,0,"['TV', 'Wifi', 'Air conditioning', 'Kitchen', ...",85.0,0.942308


In [16]:
def build_chart(df, host_is_superhost, nearby_sites_rating, bathrooms, bedrooms, beds, instant_bookable, *amenities):
    listings = df.copy()
    
    # Filter based on the condition
    listings = listings[(listings['host_is_superhost'] == host_is_superhost) & 
                    (listings['nearby_sites_rating'] >= nearby_sites_rating) & 
                    (listings['bathrooms'] >= bathrooms) & 
                    (listings['bedrooms'] >= bedrooms) & 
                    (listings['beds'] >= beds) &
                    (listings['instant_bookable'] == instant_bookable)]

    for a in amenities:
      listings = listings[listings['amenities_list'].str.contains(a)]

    q_listings = listings.sort_values('preference',ascending=False)
    
    return q_listings

In [17]:
build_chart(df_kr, 1, 2, 2, 2, 2, 0, "TV", "Air conditioning", "Kitchen").head()

,host_is_superhost,nearby_sites_rating,bathrooms,bedrooms,beds,instant_bookable,amenities_list,price,preference
6326,1,2.495057,2.0,3.0,4.0,0,"['TV', 'Cable TV', 'Wifi', 'Air conditioning',...",460.0,0.959291
6189,1,2.881733,2.0,2.0,2.0,0,"['TV', 'Cable TV', 'Internet', 'Wifi', 'Air co...",205.0,0.953954
6447,1,2.492140,2.0,2.0,2.0,0,"['TV', 'Cable TV', 'Wifi', 'Air conditioning',...",330.0,0.951515
6355,1,3.252219,2.0,3.0,4.0,0,"['TV', 'Cable TV', 'Wifi', 'Air conditioning',...",440.0,0.948269
5600,1,2.334235,2.0,2.0,2.0,0,"['TV', 'Wifi', 'Air conditioning', 'Pool', 'Ki...",188.0,0.941568


In [19]:
df_kr_new = listings[['host_is_superhost', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities_list', 'price', 'minimum_nights', 'maximum_nights',
                      'instant_bookable', 'x0_Entire home/apt', 'x0_Hotel room', 'x0_Private room', 'x0_Shared room', 'preference']]

In [20]:
def build_chart_new(df, host_is_superhost, accommodates, bathrooms, bedrooms, beds, instant_bookable, price_low, price_high, minimum_nights, maximum_nights,
                x0_Entire_home_or_apt, x0_Hotel_room, x0_Private_room, x0_Shared_room, *amenities):
    listings = df.copy()
    
    # Filter based on the condition
    listings = listings[(listings['host_is_superhost'] == host_is_superhost) & 
                    (listings['bathrooms'] >= bathrooms) & 
                    (listings['bedrooms'] >= bedrooms) & 
                    (listings['beds'] >= beds) &
                    (listings['instant_bookable'] == instant_bookable) &
                    (price_low <= listings['price']) &
                    (listings['price'] <= price_high) &
                    (listings['minimum_nights'] <= minimum_nights) &
                    (listings['maximum_nights'] >= maximum_nights) &
                    (listings['x0_Entire home/apt'] == x0_Entire_home_or_apt) &
                    (listings['x0_Hotel room'] == x0_Hotel_room) &
                    (listings['x0_Private room'] == x0_Private_room) &
                    (listings['x0_Shared room'] == x0_Shared_room)
                    ]

    for a in amenities:
      listings = listings[listings['amenities_list'].str.contains(a)]

    q_listings = listings.sort_values('preference',ascending=False)
    
    return q_listings

In [21]:
build_chart_new(df_kr_new, 1, 1, 2, 2, 2, 1, 0, 20000, 1, 2, 1, 0, 0, 0, "TV", "Kitchen").head()

,host_is_superhost,accommodates,bathrooms,bedrooms,beds,amenities_list,price,minimum_nights,maximum_nights,instant_bookable,x0_Entire home/apt,x0_Hotel room,x0_Private room,x0_Shared room,preference
5437,1,13,2.0,2.0,10.0,"['TV', 'Wifi', 'Air conditioning', 'Pool', 'Ki...",435.0,1,1125,1,1.0,0.0,0.0,0.0,0.956423
5310,1,6,2.0,2.0,2.0,"['TV', 'Cable TV', 'Wifi', 'Air conditioning',...",356.0,1,1125,1,1.0,0.0,0.0,0.0,0.943231
4620,1,6,2.0,2.0,5.0,"['TV', 'Cable TV', 'Wifi', 'Air conditioning',...",320.0,1,1124,1,1.0,0.0,0.0,0.0,0.939068
1016,1,13,6.0,6.0,8.0,"['TV', 'Wifi', 'Air conditioning', 'Pool', 'Ki...",1100.0,1,1125,1,1.0,0.0,0.0,0.0,0.934769
4625,1,6,2.0,2.0,2.0,"['TV', 'Cable TV', 'Wifi', 'Air conditioning',...",401.0,1,1125,1,1.0,0.0,0.0,0.0,0.933475


# Content-based Recommendation

In [22]:
df_cbr = listings.copy()

In [23]:
from ast import literal_eval

df_cbr['amenities_list'] = df_cbr.amenities_list.apply(literal_eval)

In [24]:
df_cbr.head(1)

,id,description,host_since,host_is_superhost,latitude,longitude,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,maximum_nights_avg_ntm,number_of_reviews_ltm,instant_bookable,calculated_host_listings_count,nearby_sites_rating,nearest_station_dist,nearest_site_dist,nearest_site_rating,host_response_time_cross_rate,email,phone,reviews,jumio,offline_government_id,selfie,government_id,identity_manual,work_email,manual_online,manual_offline,x0_Central Region,x0_East Region,x0_North Region,x0_North-East Region,x0_West Region,x0_apartment,x0_bed and breakfast,x0_boat,x0_boutique hotel,x0_bungalow,x0_condominium,x0_guest suite,x0_hostel,x0_hotel,x0_house,x0_loft,x0_other,x0_townhouse,x0_Entire home/apt,x0_Hotel room,x0_Private room,x0_Shared room,amenities_list,Wifi,Air conditioning,Kitchen,Parking,Outdoor,Kids,Pets allowed,Workspace,Gym,Breakfast,availability_average,preference
0,1101652,Explore one of Singapore's best kept heritage ...,2012-11-15,0,1.29737,103.90068,4,1.0,1.0,1.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",""Wheelc...",200.0,5,800,800.0,0,0,1,1.135049,1.81344,3.174312,4.5,2.0,1,1,1,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"[TV, Cable TV, Wifi, Air conditioning, Wheelch...",1,1,1,1,0,0,0,0,0,0,0.776522,0.894053


In [25]:
listings_with_name=pd.read_csv("/content/drive/My Drive/BT4222_Project/datasets/listings_cleaned.csv", low_memory=False)

In [26]:
listings_with_name.head(1)

,id,name,description,neighborhood_overview,host_id,host_since,host_about,host_response_time,host_response_rate,host_is_superhost,host_neighbourhood,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,nearby_sites_rating,nearest_station_dist,nearest_site_dist,nearest_site_rating
0,49091,COZICOMFORT LONG TERM STAY ROOM 2,This is Room No. 2.(available after Jan 15 201...,NaN,266763,2010-10-20,I am a private tutor by profession. My husband...,within a few hours,100%,f,Woodlands,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,f,Woodlands,North Region,1.44255,103.7958,Apartment,Private room,1,1.0,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$81.00,180,360,180,180,360,360,180.0,360.0,58 months ago,30,60,90,365,1,0,94.0,10.0,10.0,10.0,10.0,8.0,8.0,f,2,0.02,0.995731,0.617762,4.295315,4.595264


In [27]:
name_dict = {}
for index, listing in listings_with_name.iterrows():
  name_dict[listing['id']] = str(listing['name']).title()

In [28]:
df_cbr['name'] = df_cbr['id'].apply(lambda x: name_dict[x])

In [29]:
df_cbr_sample = df_cbr.iloc[:5]

In [30]:
df_cbr_sample

,id,description,host_since,host_is_superhost,latitude,longitude,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,maximum_nights_avg_ntm,number_of_reviews_ltm,instant_bookable,calculated_host_listings_count,nearby_sites_rating,nearest_station_dist,nearest_site_dist,nearest_site_rating,host_response_time_cross_rate,email,phone,reviews,jumio,offline_government_id,selfie,government_id,identity_manual,work_email,manual_online,manual_offline,x0_Central Region,x0_East Region,x0_North Region,x0_North-East Region,x0_West Region,x0_apartment,x0_bed and breakfast,x0_boat,x0_boutique hotel,x0_bungalow,x0_condominium,x0_guest suite,x0_hostel,x0_hotel,x0_house,x0_loft,x0_other,x0_townhouse,x0_Entire home/apt,x0_Hotel room,x0_Private room,x0_Shared room,amenities_list,Wifi,Air conditioning,Kitchen,Parking,Outdoor,Kids,Pets allowed,Workspace,Gym,Breakfast,availability_average,preference,name
0,1101652,Explore one of Singapore's best kept heritage ...,2012-11-15,0,1.29737,103.90068,4,1.0,1.0,1.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",""Wheelc...",200.0,5,800,800.0,0,0,1,1.135049,1.813440,3.174312,4.500000,2.00,1,1,1,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"[TV, Cable TV, Wifi, Air conditioning, Wheelch...",1,1,1,1,0,0,0,0,0,0,0.776522,0.894053,Cozy Place In Singapore
1,1299316,"Regardless your stay is as a tourists, busines...",2013-04-12,0,1.36392,103.76646,2,1.0,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",76.0,5,365,365.0,9,0,1,1.288425,0.206081,1.667353,4.500000,4.00,1,1,1,1,1,1,1,1,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,"[TV, Cable TV, Internet, Wifi, Air conditionin...",1,1,1,0,0,1,0,1,1,0,0.904224,0.893712,Private Cozy Room On The Hill
2,1731462,Right next to a shopping mall w/ Farrer Park M...,2013-08-16,0,1.31176,103.85597,2,1.0,1.0,1.0,"{Internet,Wifi,""Air conditioning"",Pool,Kitchen...",80.0,90,1125,1125.0,0,0,7,1.982270,0.210743,0.860425,4.138595,4.00,1,1,1,1,1,1,1,1,0,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,"[Internet, Wifi, Air conditioning, Pool, Kitch...",1,1,1,1,0,0,0,1,0,0,0.811644,0.792432,Cs Modern Ensuite Room In Central
3,3885720,Our spacious Studio apartment is found within ...,2011-02-07,0,1.26969,103.81375,5,1.0,0.0,3.0,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",143.0,2,1125,1125.0,22,0,1,2.550789,0.459645,0.602082,4.559654,2.70,1,1,1,1,0,0,1,0,1,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"[TV, Internet, Wifi, Air conditioning, Kitchen...",1,1,1,0,0,0,0,1,0,0,0.639574,0.817946,Studio Apartment Near Sentosa
4,4214834,Do you love coffee? Great! The apartment is ri...,2013-04-23,0,1.28358,103.83039,2,1.0,1.0,1.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Pets live...",85.0,1,1125,1125.0,4,0,1,1.689570,0.474038,1.550443,4.438634,1.72,1,1,1,1,0,0,1,0,1,0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,"[TV, Wifi, Air conditioning, Kitchen, Pets liv...",1,1,1,0,0,0,0,1,0,0,0.888261,0.942308,Cosy Room In Charming Tiong Bahru


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_2 = TfidfVectorizer(stop_words='english')

tfidf_matrix_2 = tfidf_2.fit_transform(df_cbr_sample['description'])

tfidf_matrix_2.shape

(5, 302)

In [32]:
tfidf_matrix_2.toarray()[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.08773628, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.08773628,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.07078508, 0.07078508, 0.        , 0.        , 0.08773628,
       0.        , 0.08773628, 0.        , 0.09885818, 0.17547257,
       0.        , 0.        , 0.14157016, 0.08773628, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.07078508, 0.        ,
       0.        , 0.08773628, 0.        , 0.21235524, 0.        ,
       0.        , 0.        , 0.08773628, 0.        , 0.11751602,
       0.        , 0.        , 0.        , 0.        , 0.     

In [33]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_2 = linear_kernel(tfidf_matrix_2, tfidf_matrix_2)

In [34]:
cosine_sim_2

array([[1.        , 0.05775296, 0.09518553, 0.06731624, 0.09892602],
       [0.05775296, 1.        , 0.15247382, 0.03332736, 0.09936358],
       [0.09518553, 0.15247382, 1.        , 0.02038497, 0.11327775],
       [0.06731624, 0.03332736, 0.02038497, 1.        , 0.06300565],
       [0.09892602, 0.09936358, 0.11327775, 0.06300565, 1.        ]])

In [35]:
pd.DataFrame(cosine_sim_2, index=["listing 1", "listing 2", "listing 3", "listing 4", "listing 5"], columns=["listing 1", "listing 2", "listing 3", "listing 4", "listing 5"])

,listing 1,listing 2,listing 3,listing 4,listing 5
listing 1,1.000000,0.057753,0.095186,0.067316,0.098926
listing 2,0.057753,1.000000,0.152474,0.033327,0.099364
listing 3,0.095186,0.152474,1.000000,0.020385,0.113278
listing 4,0.067316,0.033327,0.020385,1.000000,0.063006
listing 5,0.098926,0.099364,0.113278,0.063006,1.000000


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(df_cbr['description'])

tfidf_matrix.shape

(12222, 18231)

In [37]:
tfidf_matrix.toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [38]:
df_cbr.head().loc[0].description

"Explore one of Singapore's best kept heritage in the East, just steps outside this cosy 1-bedroom condo. Surpisingly quiet despite being so near to the delights of Katong with its myriad of shops and endless eateries! Explore one of Singapore's best kept heritage in the East, just steps outside this cosy 1-bedroom condo. Surpisingly quiet despite being so near to the delights of Katong with its myriad of shops and endless eateries!  Simple, cool layout and in tip top condition. Featuring a fully furnished applianced kitchen, dryer, washer and a modern bath with a rain-shower feature to destress. The Queen sized bed comfortably fits two people with additional sofa bed tucked in the living room. A small pool with jacuzzi for relaxation is available too. What is better? A good sized balcony to laze your day away drenching in the tropical sunshine!  It's a great spot to call home during your visit to Singapore! The condo is also a short walk to Parkway Parade Mall, I12Katong Mall, Katong 

In [39]:
df_cbr.head().loc[0]['name']

'Cozy Place In Singapore'

In [40]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [41]:
indices = pd.Series(df_cbr.index, index=df_cbr['name']).drop_duplicates()

In [42]:
indices

name
Cozy Place In Singapore                               0
Private Cozy Room On The Hill                         1
Cs Modern Ensuite Room In Central                     2
Studio Apartment Near Sentosa                         3
Cosy Room In Charming Tiong Bahru                     4
                                                  ...  
Common Room With Shared Toilet Best For Pnoy      12217
Kampong Living Spacious Room At Coliving Space    12218
Kampong Living Master Room At Coliving Space      12219
Kampong Living Cosy Room At Coliving Space        12220
Studio Apartment At Fort Road                     12221
Length: 12222, dtype: int64

In [43]:
def content_recommender(name, cosine_sim, df, indices):
    idx = indices[name]

    # Get the pairwsie similarity scores of all listings with that listing
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the listings based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar listings. Ignore the first one.
    sim_scores = sim_scores[1:11]

    # Get the listing indices
    listings_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar listings
    return df['name'].iloc[listings_indices]

In [44]:
content_recommender("Cozy Place In Singapore", cosine_sim, df_cbr, indices)

7570            2Br 2Bath In Katong Near F&B (Min 3 Mths)
2308                    Near To Central With Free Parking
3024     Modern 2Br Apt In Katong, Free Parking, Near F&B
3836     Humble & Conveniently Located 1Br In Katong Area
6257     Humble & Conveniently Located 1Br In Katong Area
2049                                     East Coast Haven
6336     Humble & Conveniently Located 1Br In Katong Area
12220          Kampong Living Cosy Room At Coliving Space
4369           Spacious 3Br Home With East Coast Sea View
10348    Humble & Conveniently Located 1Br In Katong Area
Name: name, dtype: object

# Metadata Based Recommender

In [45]:
listings_new = pd.read_csv("/content/drive/My Drive/BT4222_Project/final_datasets/listings_final_version2.csv")

In [46]:
df_mbr = listings_new.copy()

In [47]:
listings_with_name=pd.read_csv("/content/drive/My Drive/BT4222_Project/datasets/listings_cleaned.csv", low_memory=False)
name_dict = {}
for index, listing in listings_with_name.iterrows():
  name_dict[listing['id']] = str(listing['name']).title()

In [48]:
df_mbr['name'] = df_mbr['id'].apply(lambda x: name_dict[x])

In [49]:
df_mbr.head(1)

,id,description,host_since,host_about,host_is_superhost,host_verifications,latitude,longitude,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,maximum_nights_avg_ntm,number_of_reviews_ltm,instant_bookable,calculated_host_listings_count,nearby_sites_rating,nearest_station_dist,nearest_site_dist,nearest_site_rating,host_response_time_cross_rate,x0_Central Region,x0_East Region,x0_North Region,x0_North-East Region,x0_West Region,x0_apartment,x0_bed and breakfast,x0_boat,x0_boutique hotel,x0_bungalow,x0_condominium,x0_guest suite,x0_hostel,x0_hotel,x0_house,x0_loft,x0_other,x0_townhouse,x0_Entire home/apt,x0_Hotel room,x0_Private room,x0_Shared room,amenities_list,Wifi,Air conditioning,Kitchen,Parking,Outdoor,Kids,Pets allowed,Workspace,Gym,Breakfast,availability_average,name
0,1101652,Explore one of Singapore's best kept heritage ...,2012-11-15,Hey!\r\n\r\nWelcome to my cozy apartment . I f...,0,"['email', 'phone', 'reviews']",1.29737,103.90068,4,1.0,1.0,1.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",""Wheelc...",200.0,5,800,800.0,0,0,1,1.135049,1.81344,3.174312,4.5,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"['TV', 'Cable TV', 'Wifi', 'Air conditioning',...",1,1,1,1,0,0,0,0,0,0,0.776522,Cozy Place In Singapore


In [50]:
from ast import literal_eval

df_mbr['amenities_list'] = df_mbr.amenities_list.apply(literal_eval)

In [51]:
def sanitize(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [52]:
for feature in ['host_verifications', 'amenities_list']:
    df_mbr[feature] = df_mbr[feature].apply(sanitize)

In [53]:
df_mbr.head(1)

,id,description,host_since,host_about,host_is_superhost,host_verifications,latitude,longitude,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,maximum_nights_avg_ntm,number_of_reviews_ltm,instant_bookable,calculated_host_listings_count,nearby_sites_rating,nearest_station_dist,nearest_site_dist,nearest_site_rating,host_response_time_cross_rate,x0_Central Region,x0_East Region,x0_North Region,x0_North-East Region,x0_West Region,x0_apartment,x0_bed and breakfast,x0_boat,x0_boutique hotel,x0_bungalow,x0_condominium,x0_guest suite,x0_hostel,x0_hotel,x0_house,x0_loft,x0_other,x0_townhouse,x0_Entire home/apt,x0_Hotel room,x0_Private room,x0_Shared room,amenities_list,Wifi,Air conditioning,Kitchen,Parking,Outdoor,Kids,Pets allowed,Workspace,Gym,Breakfast,availability_average,name
0,1101652,Explore one of Singapore's best kept heritage ...,2012-11-15,Hey!\r\n\r\nWelcome to my cozy apartment . I f...,0,"['email','phone','reviews']",1.29737,103.90068,4,1.0,1.0,1.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",""Wheelc...",200.0,5,800,800.0,0,0,1,1.135049,1.81344,3.174312,4.5,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"[tv, cabletv, wifi, airconditioning, wheelchai...",1,1,1,1,0,0,0,0,0,0,0.776522,Cozy Place In Singapore


In [54]:
def create_soup(x):
    return x['description'] + ' ' + str(x['host_about']) + ' '+ ' '.join(x['host_verifications']) + ' ' + ' '.join(x['amenities_list'])

In [55]:
df_mbr['soup'] = df_mbr.apply(create_soup, axis=1)

In [56]:
df_mbr.iloc[0]['soup']

"Explore one of Singapore's best kept heritage in the East, just steps outside this cosy 1-bedroom condo. Surpisingly quiet despite being so near to the delights of Katong with its myriad of shops and endless eateries! Explore one of Singapore's best kept heritage in the East, just steps outside this cosy 1-bedroom condo. Surpisingly quiet despite being so near to the delights of Katong with its myriad of shops and endless eateries!  Simple, cool layout and in tip top condition. Featuring a fully furnished applianced kitchen, dryer, washer and a modern bath with a rain-shower feature to destress. The Queen sized bed comfortably fits two people with additional sofa bed tucked in the living room. A small pool with jacuzzi for relaxation is available too. What is better? A good sized balcony to laze your day away drenching in the tropical sunshine!  It's a great spot to call home during your visit to Singapore! The condo is also a short walk to Parkway Parade Mall, I12Katong Mall, Katong 

In [57]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df_mbr['soup'])

from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

df_mbr = df_mbr.reset_index()
indices2 = pd.Series(df_mbr.index, index=df_mbr['name'])

In [58]:
def content_recommender(name, cosine_sim, df, indices):
    idx = indices[name]

    # Get the pairwsie similarity scores of all listings with that listing
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the listings based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar listings. Ignore the first one.
    sim_scores = sim_scores[1:11]

    # Get the listing indices
    listings_indices = [i[0] for i in sim_scores]

    df_new = pd.DataFrame(df['name'].iloc[listings_indices])
    df_new = df_new.assign(scores = list(map(lambda x: x[1], sim_scores)))

    # Return the top 10 most similar listings
    return df_new

In [59]:
content_recommender("Cozy Place In Singapore", cosine_sim2, df_mbr, indices2)

,name,scores
507,1 Br W/ Prvt. Bathroom Available In Heart Of Cbd!,0.410496
3610,Comfy Room @Jurongeast,0.402591
2269,"Mins To Cbd, 2 Br Apartment",0.402432
3382,Huge 2Bdrm >5Mins To Paya Lebar Mrt,0.401115
3775,Discount Bright Homey 2Br Near City Peranakanarea,0.393228
4307,Discount Bright Quiet 2Br Big Balcony Near City,0.372484
2918,Raffles Place Sea View 1Br Condo,0.369175
7476,Central 2Br Apt In Chinatown Above Mrt (#69),0.368142
6262,1Br Suite In Chinatown/Cbd Above Mrt (#65),0.366119
7520,"Spacious 1+Study Above Mrt Station, (P-3)",0.365869


In [60]:
def recommend(name):
  return content_recommender(name, cosine_sim, df_cbr, indices)

In [61]:
recommend("Cozy Place In Singapore")

,name,scores
7570,2Br 2Bath In Katong Near F&B (Min 3 Mths),0.200835
2308,Near To Central With Free Parking,0.194561
3024,"Modern 2Br Apt In Katong, Free Parking, Near F&B",0.175625
3836,Humble & Conveniently Located 1Br In Katong Area,0.172282
6257,Humble & Conveniently Located 1Br In Katong Area,0.172282
2049,East Coast Haven,0.169960
6336,Humble & Conveniently Located 1Br In Katong Area,0.169191
12220,Kampong Living Cosy Room At Coliving Space,0.168525
4369,Spacious 3Br Home With East Coast Sea View,0.167057
10348,Humble & Conveniently Located 1Br In Katong Area,0.165239


In [62]:
content_recommender("Cozy Place In Singapore", cosine_sim, df_cbr, indices)

,name,scores
7570,2Br 2Bath In Katong Near F&B (Min 3 Mths),0.200835
2308,Near To Central With Free Parking,0.194561
3024,"Modern 2Br Apt In Katong, Free Parking, Near F&B",0.175625
3836,Humble & Conveniently Located 1Br In Katong Area,0.172282
6257,Humble & Conveniently Located 1Br In Katong Area,0.172282
2049,East Coast Haven,0.169960
6336,Humble & Conveniently Located 1Br In Katong Area,0.169191
12220,Kampong Living Cosy Room At Coliving Space,0.168525
4369,Spacious 3Br Home With East Coast Sea View,0.167057
10348,Humble & Conveniently Located 1Br In Katong Area,0.165239


In [63]:
def recommend(name):
  return content_recommender(name, cosine_sim2, df_mbr, indices2)

In [64]:
recommend("Cozy Place In Singapore")

,name,scores
507,1 Br W/ Prvt. Bathroom Available In Heart Of Cbd!,0.410496
3610,Comfy Room @Jurongeast,0.402591
2269,"Mins To Cbd, 2 Br Apartment",0.402432
3382,Huge 2Bdrm >5Mins To Paya Lebar Mrt,0.401115
3775,Discount Bright Homey 2Br Near City Peranakanarea,0.393228
4307,Discount Bright Quiet 2Br Big Balcony Near City,0.372484
2918,Raffles Place Sea View 1Br Condo,0.369175
7476,Central 2Br Apt In Chinatown Above Mrt (#69),0.368142
6262,1Br Suite In Chinatown/Cbd Above Mrt (#65),0.366119
7520,"Spacious 1+Study Above Mrt Station, (P-3)",0.365869


# Collaborative Filtering Recommendation

In [65]:
!pip install surprise

     |████████████████████████████████| 11.8MB 5.6MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617542 sha256=efd976267fb409c3535893f9ea20a5dd7c3c8954ab06d170d445b71ca4b2e402
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [66]:
from surprise import SVD, Reader, Dataset
from surprise.model_selection import KFold

In [67]:
df_cfr = customers.copy()

In [68]:
counts = df_cfr['listing_id'].value_counts()
df_cfr = df_cfr[df_cfr['listing_id'].isin(counts[counts >= 10].index)]

In [69]:
counts1 = df_cfr['reviewer_id'].value_counts()
df_cfr = df_cfr[df_cfr['reviewer_id'].isin(counts1[counts1 > 2].index)]

In [70]:
counts1 = df_cfr['reviewer_id'].value_counts()
df_cfr[df_cfr['reviewer_id'].isin(counts1[counts1 == 1].index)].shape

(0, 14)

In [71]:
df_cfr.shape

(7483, 14)

In [72]:
df = df_cfr[['reviewer_id','listing_id','polarity_score' ]]
df.head()
print(df.nunique())

reviewer_id       1878
listing_id        2049
polarity_score    1819
dtype: int64


In [73]:
df.head(10).reset_index().drop("index", axis=1)

,reviewer_id,listing_id,polarity_score
0,17709890,50646,0.9802
1,4848541,71896,0.9868
2,1369617,241503,0.9781
3,1401333,241503,0.9600
4,2302404,241503,0.9337
5,2506808,241503,0.9902
6,5125575,241503,0.9933
7,2506808,241503,0.9431
8,7713647,241503,0.9943
9,13588897,241503,0.9842


In [74]:
reader = Reader(rating_scale=(-1, 1))
data = Dataset.load_from_df(df[['reviewer_id', 'listing_id', 'polarity_score']], reader)

In [75]:
raw_ratings = data.raw_ratings

import random
from surprise.model_selection import GridSearchCV
# shuffle ratings if you want
random.shuffle(raw_ratings)

# A = 90% of the data, B = 10% of the data
threshold = int(.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # data is now the set A

# Select your best algo with grid search.
print('Grid Search...')

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005,0.1],
              'reg_all': [0.4, 0.6], 'n_factors':[100,500]} # tuning
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
grid_search.fit(data)
print('best RMSE score')
print(grid_search.best_score['rmse'])

Grid Search...
best RMSE score
0.2514488397393215


In [76]:
print('combination of parameters that gave the best RMSE score')
print(grid_search.best_params['rmse'])
algo = grid_search.best_estimator['rmse']

combination of parameters that gave the best RMSE score
{'n_epochs': 5, 'lr_all': 0.1, 'reg_all': 0.6, 'n_factors': 100}


In [77]:
# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

from surprise import accuracy
# Compute biased accuracy on A
testset= trainset.build_testset()
predictions = algo.test(testset)
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions,verbose=True)
print('len(predictions)')
print(len(predictions))

Biased accuracy on A,   RMSE: 0.1600
len(predictions)
6734


In [79]:
algo.pu.shape

(1877, 100)

In [80]:
# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)
print('len(predictions)')
print(len(predictions))

Unbiased accuracy on B, RMSE: 0.2608
len(predictions)
749


In [82]:
# define a cross-validation iterator
kf = KFold(n_splits=7)
algo = SVD(n_factors=500, n_epochs=5, lr_all=0.1)

for trainset, testset in kf.split(data):
    # train and test algorithm.
    algo.fit(trainset)
    
    predictions = algo.test(testset)
    
    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)


RMSE: 0.2987
RMSE: 0.2964
RMSE: 0.2689
RMSE: 0.3080
RMSE: 0.2698
RMSE: 0.3129
RMSE: 0.2912


In [83]:
# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

testset= trainset.build_testset() # anti?
predictionsAll = algo.test(testset)
print('Accuracy on whole data set,', end='   ')
accuracy.rmse(predictionsAll,verbose=True)
print('len(predictions)')
print(len(predictionsAll))

Accuracy on whole data set,   RMSE: 0.0945
len(predictions)
6734


In [85]:
def get_top3_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]    
    return top_recs
print('getting top 3 recommendations')

getting top 3 recommendations


In [86]:
from collections import defaultdict
top3_recommendations = get_top3_recommendations(predictionsAll,3)

print('Total predictions calculated are %d' % len(predictionsAll))

Total predictions calculated are 6734


In [87]:
top3_recommendations.items()

dict_items([(5843865, [(12475063, 0.9889417732443291), (12345284, 0.13470683359035984)]), (17273265, [(26464626, 0.9712978521973402), (17813122, 0.964823612659556), (13462899, 0.9628447441227352)]), (49304010, [(7350502, 0.8161682025385393), (5739284, 0.7974365535519635), (12859055, 0.6253156804838492)]), (582032, [(6686851, 0.9623501110700715), (7931127, 0.947646195263257), (11267014, 0.8649145708083287)]), (3290533, [(6117032, 0.9809559698233357), (7622220, 0.9730713227921345), (3988536, 0.9122377487422269)]), (22750305, [(15438033, 0.9517538423038701), (2731054, 0.9387006745192152), (21430677, 0.921579578418322)]), (20613825, [(6431505, 0.9268069359164662), (6431505, 0.9268069359164662), (29830408, 0.7404078346226195)]), (9897832, [(19687504, 0.7842728796039883), (7843168, 0.7217338424641726), (16502098, 0.6392361468740102)]), (125653187, [(21154795, 0.8835974368564046), (16329592, 0.8476389235413251), (16329592, 0.8476389235413251)]), (174869022, [(12924764, 0.8825876283706559), (1

In [88]:
pd.DataFrame(top3_recommendations[29084250], columns = ["Recommended Listing Id", "Score"])

,Recommended Listing Id,Score
0,4541183,0.964334
1,25328461,0.933863
2,37655038,0.750542


In [89]:
def recommend(reviewer_id): 
  return pd.DataFrame(top3_recommendations[reviewer_id], columns = ["Recommended Listing Id", "Score"])

In [90]:
recommend(29084250)

,Recommended Listing Id,Score
0,4541183,0.964334
1,25328461,0.933863
2,37655038,0.750542


In [93]:
dfo = pd.DataFrame(columns=['UserId', 'Recommended Listing,Rating'])
i=0;
for uid, user_ratings in top3_recommendations.items():
    row = [uid, top3_recommendations[uid]]
    dfo.loc[i] = row
    i=i+1
dfo.to_csv('/content/drive/My Drive/BT4222_Project/submission.csv', index = False)